### Libraries

In [1]:
# packages to install

from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import requests
import re
from datetime import datetime
import pathlib


In [2]:
url = 'https://ecf.dcd.uscourts.gov/cgi-bin/CourtSched.pl'
page = requests.get(url)



<Response [200]>


In [3]:
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
table = soup.find("table", {"id":"ts"})
date = soup.find("b").text.split(":")[1]
date = " ".join(date.split())
formatted_date = datetime.strptime(date, '%A, %B %d, %Y')


datetime.datetime(2022, 7, 21, 0, 0)

In [5]:
ls_table_tr = table.find_all("tr")
rows = []
for tr in ls_table_tr:
    row = []
    count = 0
    for child in tr.children:
        if count == 0:
            result = child.text.replace('\n', '')
            if result != "":
                 row.append(result)
        else: 

            try:
                row.append(child.text.replace('\n', ''))
            except:
                continue

    count = count + 1
    if len(row) > 0:
        rows.append(row)
df = pd.DataFrame(rows[1:], columns = rows[0])

In [6]:
df[['Case Number', 'Title']] = df['Case Number and Title'].str.split(':', expand=True)

In [7]:
df["Date"] = formatted_date

In [15]:
df

,Case Number and Title,Judge,Time,Courtroom,Purpose,Case Number,Title,Date
0,21-cr-0204: USA v. BLEDSOE,Chief Judge Beryl A. Howell,09:15AM,Courtroom 22A- In Person,Jury Trial,21-cr-0204,USA v. BLEDSOE,2022-07-21
1,21-cv-1567: TARQUINII v. HARKER,Judge Rudolph Contreras,10:00AM,Telephonic/VTC,Status Conference,21-cv-1567,TARQUINII v. HARKER,2022-07-21
2,19-cr-0376: USA v. CORDOVA,Judge James E. Boasberg,10:30AM,Courtroom 25A- In Person,Status Conference,19-cr-0376,USA v. CORDOVA,2022-07-21
3,21-cr-0708: USA v. KELLY,Judge Royce C. Lamberth,12:45PM,Telephonic/VTC,Status Conference,21-cr-0708,USA v. KELLY,2022-07-21
4,21-cr-0099: USA v. GORDON,Judge Paul L. Friedman,01:00PM,Telephonic/VTC,Status Conference,21-cr-0099,USA v. GORDON,2022-07-21
5,22-cr-0244: USA v. SPEED,Mag. Judge G. M. Harvey,01:00PM,Telephonic/VTC,Preliminary Hearing,22-cr-0244,USA v. SPEED,2022-07-21
6,21-cr-0722: USA v. MOONEY-RONDON,Mag. Judge G. M. Harvey,01:00PM,Telephonic/VTC,Status Conference,21-cr-0722,USA v. MOONEY-RONDON,2022-07-21
7,22-cr-0070: USA v. DENNEY,Judge Randolph D. Moss,01:00PM,Courtroom 8- In Person,Sentencing,22-cr-0070,USA v. DENNEY,2022-07-21
8,21-cr-0271: USA v. BANCROFT,Judge Emmet G. Sullivan,01:00PM,Telephonic/VTC,Sentencing,21-cr-0271,USA v. BANCROFT,2022-07-21
9,21-cr-0271: USA v. SANTOS-SMITH,Judge Emmet G. Sullivan,01:00PM,Telephonic/VTC,Sentencing,21-cr-0271,USA v. SANTOS-SMITH,2022-07-21


In [18]:

file_date = formatted_date.strftime("%Y-%m-%d")
path = pathlib.Path(f"../data/days/{file_date}-court_calendar.csv")
df.to_csv(path, index = False)